In [5]:
from defines import *
from model import *
from data import *
import sys

from numpy import loadtxt
from keras.models import load_model
from PIL import Image

import glob
import os 
import math
import shutil

In [9]:
PARAM_BETA_TEST_NUM = 0

In [11]:
path = 'data_jupyter'# main folder path

D = 'all_images'
L = 'all_labels'

D_P_im = 'polar_im'
D_C_im = 'car_im'
D_P = 'polar_l'
D_C = 'car_l'

prev_number_of_D_P = -1
prev_number_of_D_C = -1

diff = math.inf

model_P = unet(PARAM_BETA1[PARAM_BETA_TEST_NUM], PARAM_BETA2[PARAM_BETA_TEST_NUM])
model_C = unet(PARAM_BETA1[PARAM_BETA_TEST_NUM], PARAM_BETA2[PARAM_BETA_TEST_NUM])

In [ ]:
#fill in the weights here
model_P.load_weights() 
model_C.load_weights() 

In [ ]:
count = 0

#change file name of D, L
for count, filename in enumerate(os.listdir(path+'/'+D)): 
    dst = str(count) + ".png"
    src = path+'/'+D+'/'+filename
    dst = path+'/'+D+'/'+dst

    os.rename(src, dst) 
    
for count, filename in enumerate(os.listdir(path+'/'+L)): 
    dst = str(count) + ".tif"
    src = path+'/'+L+'/'+filename
    dst = path+'/'+L+'/'+dst

    os.rename(src, dst) 

In [12]:
def dscore(im, im_name):
    label = Image.open(os.path.join(L, im_name))
    
    pixelIm = im.load()
    pixelLabel = label.load()
    
    upper = 0
    lower = im.size[0] * im.size[1]
    
    for i in range(im.size[0]):
        for j in range(im.size[1]):
            if pixelIm[i,j] == pixelLabel[i,j]:
                upper = upper + 1
            
    upper = upper * 2
    
    return upper / lower
                

In [ ]:
while (diff > 0): 
    #load model
    #if count > 0:
    #    model_P.load_weights('model_P_weights_' + (count - 1))
    #    model_C.load_weights('model_C_weights_' + (count - 1))
    
    count = count + 1
    
    testGene_P, testGene_C = testGenerator(path,D,L)
    
    #perdict
    results_P = model_P.predict(testGene_P, PARAM_N_TESTS, verbose=1)
    results_C = model_C.predict(testGene_C, PARAM_N_TESTS, verbose=1)

    np.save(PARAM_PATH_TEST_NPY_P, results_P)
    saveResult(PARAM_PATH_TEST_RESULTS_P, results_P)
    np.save(PARAM_PATH_TEST_NPY_C, results_C)
    saveResult(PARAM_PATH_TEST_RESULTS_C, results_C)
    
    mergeIm(path, D, L, PARAM_PATH_TEST_RESULTS_P, path+'/'+D_P)
    mergeIm(path, D, L, PARAM_PATH_TEST_RESULTS_C, path+'/'+D_C)
    
    #change file name of D_P, D_C
    for count, filename in enumerate(os.listdir(path+'/'+D_P)): 
        dst = str(count) + ".tif"
        src = path+'/'+D_P+'/'+filenaLme
        dst = path+'/'+D_P+'/'+dst

        os.rename(src, dst) 
  
    for count, filename in enumerate(os.listdir(path+'/'+D_C)): 
        dst = str(count) + ".tif"
        src = path+'/'+D_C+'/'+filename
        dst = path+'/'+D_C+'/'+dst

        os.rename(src, dst) 
    
    #find the better one (based on L) and modify D_P, D_P_im, D_C, D_C_im
    for file in os.listdir(D):
        im_P = Image.open(path+'/'+D_P+'/'+file)
        im_C = Image.open(path+'/'+D_C+'/'+file)
        
        if dscore(im_P, file) > dscore(im_C, file):
            shutil.copyfile(path+'/'+D+'/'+file, path+'/'+D_P_im+'/'+file)
            os.remove(path+'/'+D_C+'/'+file)
        else:
            shutil.copyfile(path+'/'+D+'/'+file, path+'/'+D_C_im+'/'+file)
            os.remove(path+'/'+D_P+'/'+file)
            
    number_of_D_P = len(glob.glob(D_P))
    number_of_D_C = len(glob.glob(D_C))

    # file numbers difference
    diff = Math.abs(prev_number_of_D_P - number_of_D_P + prev_number_of_D_C - number_of_D_C) / 2
    
    prev_number_of_D_P = number_of_D_P
    prev_number_of_D_C = number_of_D_P
    
    #train model_P only on D_P, model_C only on D_C
    myGene_P = trainGenerator(PARAM_BATCHES, 
                            path, 
                            D_P, 
                            D_P_im, 
                            PARAM_DATA_ARGS, 
                            save_to_dir = PARAM_AUG_FOLDER_P)
    
    myGene_C = trainGenerator(PARAM_BATCHES, 
                            path, 
                            D_C, 
                            D_C_im, 
                            PARAM_DATA_ARGS, 
                            save_to_dir = PARAM_AUG_FOLDER_C)
    
    model_checkpoint_P = ModelCheckpoint( PARAM_SAVED_MODEL, 
                                         monitor = PARAM_METRICS, 
                                         verbose = 1, 
                                         save_best_only = PARAM_SAVE_BEST_ONLY)
    
    model_P.fit_generator(myGene_P,
                        steps_per_epoch = PARAM_EPOCH_STEPS,
                        epochs = PARAM_N_EPOCHS,
                        callbacks = [model_checkpoint])
    model_C.fit_generator(myGene_C,
                        steps_per_epoch = PARAM_EPOCH_STEPS,
                        epochs = PARAM_N_EPOCHS,
                        callbacks = [model_checkpoint])
    
    model_P.save_weights('model_P_weights_' + count)
    model_C.save_weights('model_C_weights_' + count)
    
    shutil.rmtree(path+'/'+D_P)
    shutil.rmtree(path+'/'+D_C)
    shutil.rmtree(path+'/'+D_P_im)
    shutil.rmtree(path+'/'+D_C_im)
    

save_model(model_P, 'model_P.h5')
save_model(model_C, 'model_C.h5')
    